<a href="https://colab.research.google.com/github/erfan-sams/A_star-and-uniform-cost-search-algorithm/blob/main/lin_alg_simplex_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from IPython.display import display, HTML
np.seterr(divide='ignore')
np.set_printoptions(precision=5, suppress=True,edgeitems=9, linewidth=200)
np.core.arrayprint._line_width = 180

In [ ]:
def convert_to_float(frac_str):
    try:
        return float(frac_str)
    except ValueError:
        num, denom = frac_str.split('/')
        try:
            leading, num = num.split(' ')
            whole = float(leading)
        except ValueError:
            whole = 0
        frac = np.true_divide(float(num), float(denom))
        return whole - frac if whole < 0 else whole + frac

In [ ]:
def parser(text):
  X = dict()
  s_text = text.split()
  for i in range(0, len(s_text)-1, 2):
    snt = s_text[i]
    f_var = snt.find(next(filter(str.isalpha, snt)))
    if f_var == 0:
      tmp = '1'
    else:
      tmp = snt[:f_var]
      
    var = snt[f_var:]
    if i > 0:
      X[var] = (convert_to_float(s_text[i-1]+tmp))
    else:
      if tmp == '-' or tmp == '+':
        tmp += '1'
      X[var] = (convert_to_float(tmp))
    
  X['right_value'] = (convert_to_float(s_text[-1]))

  return X

In [ ]:
def x_matrix(constraints_list):
  X_list = list()
  for cons in constraints_list:
    tmp = parser(cons)
    X_list.append(tmp)
  return X_list

In [ ]:
def row_guess(arg_col,x):  
  left_val = x.iloc[1:,arg_col].values
  devide_val = np.true_divide(x['right_value'].values[1:], left_val)
  arg_row = np.where(left_val > 0, devide_val, np.inf).argmin() + 1
  arg_rval = x.iloc[arg_row,arg_col]
  if arg_rval > 0:
    return (arg_col, arg_row)
  return 'infinite result! NBV: ' + str(x.columns[arg_col])

In [ ]:
def guess(x, status):
  bv = x.index[1: ]
  if status == 'max':
    arg_col = np.argmin(x.values[0][1:-1]) + 1
    if x.values[0][arg_col] < 0:
      return row_guess(arg_col, x)
  else:
    arg_col = np.argmax(x.values[0][1:-1]) + 1
    if x.values[0][arg_col] > 0:    
      return row_guess(arg_col, x)
  return 'the end!'

In [ ]:
def calculate(x, col,row):
  c = x.astype("float")
  c[row] = c[row] / c[row][col]
  for i in range(x.shape[0]):
    if i == row:
      continue
    c[i] = c[i] + (-1 * c[i][col]) * c[row]
  return c

In [ ]:
def create_table(constraints_list, variables, BV):
  columns_order= ['z'] + variables + ['right_value']
  index_name = ['z'] + BV
  try:
    X_list = x_matrix(constraints_list)
    # df = pd.DataFrame(X_list, index=index_name).fillna(0)
    df = pd.DataFrame(X_list).fillna(0)
  except:
    print('index:',index_name, 'columns:', variables)
    return 'please check your input format.'
  df = df.reindex(columns=columns_order)
  return df

In [ ]:
def sim_alg(df, variables, BV):
  p_df = df.copy()
  Ct_bv = p_df.iloc[0][BV].values
  B_inv = np.linalg.inv(p_df[BV].drop(0))

  for var in variables:
    df[var][1:] = B_inv @ p_df[var][1:].values                    # a bar
    df[var][0] = np.subtract(Ct_bv @ df[var][1:].values, p_df[var][0]) # C bar
  df['right_value'][1:] = B_inv @ p_df['right_value'][1:]         # b bar
  df['right_value'][0] = Ct_bv @ df['right_value'][1:]            # z bar

  return df

In [ ]:
def index_rename(df):  
  index_name = ['z'] + BV
  keys_list = df.index.values
  values_list = index_name
  zip_iterator = zip(keys_list, values_list)
  index_dict = dict(zip_iterator)
  df = df.rename(index=index_dict)
  return df

In [ ]:
def main(constraints_list, variables, BV, status):
  zero_df = create_table(constraints_list, variables, BV)
  df = sim_alg(zero_df, variables, BV)
  df = index_rename(df)
  df[np.abs(df) < np.finfo(np.float).eps] = 0
  display(df)
  while True:
    t = guess(df, status)
    if type(t) != tuple:
      print(t)
      return df
    print('incoming variable: {}, outgoing variable: {}'.format(df.columns[t[0]], df.index[t[1]]))
    df.values[:] = calculate(df.values, t[0], t[1])
    df.rename(index={df.index[t[1]]: df.columns[t[0]]}, inplace=True)
    print('\n','-------'* (len(variables)+2) ,'\n')
    display(df)

## question 2 of Linear Optimization exam!

<img src = 'https://drive.google.com/uc?id=1U-FCb2mhJS0HmASk_b15GURtKxTv0DPv'>

In [ ]:
# question 2
constraints_list = [
                    'z + 2x1 + x2 + 4x3 - 5x4 = 0',
                    '3x1 + 6x2 + 4x3 + 2x4 - e1 = 60',
                    ]
status = 'min'
variables = ['x1', 'x2', 'x3', 'x4', 'e1'] # order of variables
BV = ['x3']

df = main(constraints_list, variables, BV, status)

,z,x1,x2,x3,x4,e1,right_value
z,1.0,1.00,5.0,0.0,7.0,-1.00,60.0
x3,0.0,0.75,1.5,1.0,0.5,-0.25,15.0


incoming variable: x4, outgoing variable: x3

 ------------------------------------------------- 



,z,x1,x2,x3,x4,e1,right_value
z,1.0,-9.5,-16.0,-14.0,0.0,2.5,-150.0
x4,0.0,1.5,3.0,2.0,1.0,-0.5,30.0


infinite result! NBV: e1


##First question from LO_Chapter3 pdf

In [ ]:
constraints_list = [
                    'z + 60x1 + 30x2 + 20x3 = 0',
                    '8x1 + 6x2 + x3 + s1 = 48',
                    '4x1 + 2x2 + 3/2x3 + s2 = 20',
                    '2x1 + 3/2x2 + 1/2x3 + s3 = 8',
                    'x2 + s4 = 5'
                    ]
status = 'max'
variables = ['x1', 'x2', 'x3', 's1', 's2', 's3','s4'] # order of variables
BV = ['s1','s2','s3','s4']

df = main(constraints_list, variables, BV, status)

,z,x1,x2,x3,s1,s2,s3,s4,right_value
z,1.0,-60.0,-30.0,-20.0,0.0,0.0,0.0,0.0,0.0
s1,0.0,8.0,6.0,1.0,1.0,0.0,0.0,0.0,48.0
s2,0.0,4.0,2.0,1.5,0.0,1.0,0.0,0.0,20.0
s3,0.0,2.0,1.5,0.5,0.0,0.0,1.0,0.0,8.0
s4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,5.0


incoming variable: x1, outgoing variable: s3

 --------------------------------------------------------------- 



,z,x1,x2,x3,s1,s2,s3,s4,right_value
z,1.0,0.0,15.00,-5.00,0.0,0.0,30.0,0.0,240.0
s1,0.0,0.0,0.00,-1.00,1.0,0.0,-4.0,0.0,16.0
s2,0.0,0.0,-1.00,0.50,0.0,1.0,-2.0,0.0,4.0
x1,0.0,1.0,0.75,0.25,0.0,0.0,0.5,0.0,4.0
s4,0.0,0.0,1.00,0.00,0.0,0.0,0.0,1.0,5.0


incoming variable: x3, outgoing variable: s2

 --------------------------------------------------------------- 



,z,x1,x2,x3,s1,s2,s3,s4,right_value
z,1.0,0.0,5.00,0.0,0.0,10.0,10.0,0.0,280.0
s1,0.0,0.0,-2.00,0.0,1.0,2.0,-8.0,0.0,24.0
x3,0.0,0.0,-2.00,1.0,0.0,2.0,-4.0,0.0,8.0
x1,0.0,1.0,1.25,0.0,0.0,-0.5,1.5,0.0,2.0
s4,0.0,0.0,1.00,0.0,0.0,0.0,0.0,1.0,5.0


the end!


In [ ]:
# Exercise 6

constraints_list = [
                    'z - x1 + 2x2 = 0',
                    '3x1 + 4x2 - e1 = 30',
                    '-x1 + 2x2 + s2 = 10',
                    ]
status = 'min' # max
variables = ['x1', 'x2', 's2', 'e1'] # order of variables
BV = ['x1','x2']

df = main(constraints_list, variables, BV, status)

,z,x1,x2,s2,e1,right_value
z,1.0,0.0,0.0,1.0,0.0,10.0
x1,0.0,1.0,0.0,-0.4,-0.2,2.0
x2,0.0,0.0,1.0,0.3,-0.1,6.0


incoming variable: s2, outgoing variable: x2

 ------------------------------------------ 



,z,x1,x2,s2,e1,right_value
z,1.0,0.0,-3.333333,0.0,0.333333,-10.0
x1,0.0,1.0,1.333333,0.0,-0.333333,10.0
s2,0.0,0.0,3.333333,1.0,-0.333333,20.0


infinite result! NBV: e1
